In [ ]:
import cv2
import pickle
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.models import load_model
from keras.layers import Dropout, Flatten, Dense
from keras import optimizers
from keras.applications import vgg19, inception_v3, resnet50
from sklearn.model_selection import train_test_split

In [ ]:
num_classes = 40
epochs = 1
batch_size = 12

# Data preprocessing

**NOTE:** Do not repickle unless needed

In [ ]:
raw = [cv2.imread(x) for x in glob("data/train/*")]
raw

In [ ]:
def transform(img):
    canvas = np.full((800,800,3), (255, 255, 255), np.uint8)
    while(img.shape[0] > 800 or img.shape[1] > 800):
        img = cv2.resize(img, (img.shape[0]//2, img.shape[1]//2))
    canvas[:img.shape[0], :img.shape[1]] = img
    return canvas
x = np.array([transform(i) for i in raw])
del raw
pickle.dump(x, open("transformed.pkl", "wb"), protocol=4)

In [ ]:
x = pickle.load(open("transformed.pkl", 'rb'))

In [ ]:
plt.imshow(x[0])

In [ ]:
df = pd.read_csv("data/train.csv")
df.head()

In [ ]:
train_names = [x[11:][:-4] for x in glob("data/train/*")]
y = [df["breedID"][int(i)] for i in train_names]
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
del x

# Loading and training model

In [ ]:
model = load_model("vgg19.h5")

In [ ]:
opt = optimizers.SGD(lr=0.001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs,shuffle=True, validation_data=(x_test, y_test))